In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow


from sklearn.model_selection import train_test_split, GridSearchCV


import sys
sys.path.append('../')
from scripts import featurizer_rn

2022-08-09 21:50:41.324334: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-09 21:50:41.327645: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-09 21:50:41.327656: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# load dataset
df = pd.read_csv("../../data/dfff.csv", low_memory=False)


In [3]:
df.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,D_43,D_44,B_4,...,R_28,D_139,D_140,D_144,D_145,customer_ID,S_2,D_63,D_64,target
0,0.947242,0.002204,0.008002,1.007516,0.002830,0.453227,0.005992,NaN,0.001613,0.029088,...,0.000518,0.003183,0.005264,0.004306,0.002719,743d55408505dee752adc6e62b273f4397db6e89f19034...,2017-08-10,CO,O,0
1,0.757648,0.002076,0.008751,0.814083,0.000408,0.155700,0.004156,0.127285,0.129051,0.040101,...,0.001186,1.000928,0.005655,0.007953,0.095926,522c85a354bfec790067b54e9db7353d20d4709ae5dc93...,2017-07-04,CO,O,0
2,0.392041,0.005770,0.029682,0.810262,0.006672,0.180378,0.005028,NaN,0.126794,0.210560,...,0.006967,NaN,NaN,NaN,NaN,4488928018c20bf6261e69c8a94a47b6206a2fe91480b8...,2017-07-29,CR,NaN,1
3,0.673978,0.588604,0.295201,0.024650,0.007951,0.125583,0.008760,NaN,0.005867,0.175950,...,0.000805,0.003558,0.008268,0.003020,0.002798,9296e983c3e4f99b046952397fc4ba950abf1032a9bd6c...,2017-03-30,CO,R,1
4,0.778075,0.006930,0.056721,0.819369,0.004922,0.089214,0.006884,0.041515,0.002841,0.001179,...,0.009053,0.004670,0.000125,0.000819,0.000342,3fa10314686e0e51d8f76235eceab91d70aa3f24a98c3e...,2018-01-17,CL,O,0


In [ ]:
X, Y = featurizer_rn.featurizer(df)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [5]:
X.shape

(276572, 128)

In [6]:
dataset = X.values

In [7]:
X = dataset.astype(float)
y = Y

In [8]:
kfold = StratifiedKFold(n_splits=10, shuffle=True)

In [9]:
# baseline model
def create_baseline(input_dim):
    # create model
    model = Sequential()
    model.add(Dense(120, input_dim=input_dim, activation='relu'))
    model.add(Dense(400, activation='relu'))
    model.add(Dense(400, activation='relu'))
    model.add(Dense(400, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [10]:
for train, test in kfold.split(X, y):
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

In [11]:
model_baseline = create_baseline(X_train.shape[1])

2022-08-09 21:46:44.625893: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-09 21:46:44.625909: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-09 21:46:44.625930: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (latitude5421): /proc/driver/nvidia/version does not exist
2022-08-09 21:46:44.626102: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
model_baseline.fit(X_train, y_train, epochs=10, verbose=1, batch_size=10,validation_split=0.2)

Epoch 1/10
19914/19914 [==============================] - 25s 1ms/step - loss: 0.5644 - accuracy: 0.7513 - val_loss: 0.5620 - val_accuracy: 0.7503
Epoch 2/10
19914/19914 [==============================] - 25s 1ms/step - loss: 0.5610 - accuracy: 0.7513 - val_loss: 0.5620 - val_accuracy: 0.7503
Epoch 3/10
19914/19914 [==============================] - 25s 1ms/step - loss: 0.5610 - accuracy: 0.7513 - val_loss: 0.5620 - val_accuracy: 0.7503
Epoch 4/10
19914/19914 [==============================] - 25s 1ms/step - loss: 0.5610 - accuracy: 0.7513 - val_loss: 0.5620 - val_accuracy: 0.7503
Epoch 5/10
19914/19914 [==============================] - 25s 1ms/step - loss: 0.5610 - accuracy: 0.7513 - val_loss: 0.5620 - val_accuracy: 0.7503
Epoch 6/10
10148/19914 [==============>...............] - ETA: 11s - loss: 0.5602 - accuracy: 0.7520